In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score 
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import accuracy_score

In [2]:
df = pd.read_csv('Cleaned Data with States.csv')
df.head()

,Unnamed: 0,Alcohol,Appellation,Category,Points,Price,Review,Wine,Winery,Year,Variety,State
0,0,14.9,"Russian River Valley, Sonoma, California, US",Red,87,42.00,"Baked plum, licorice and lavender aromas and f...",V. Sattui 2015 Gilsson Vineyard Old Vine Zinfa...,V. Sattui,2015.0,Zinfandel,California
1,1,14.5,"Central Coast, Central Coast, California, US",Red,87,19.98,Pomegranate and light baking-spice aromas show...,Wente 2014 Coastal Selection Pinot Noir (Centr...,Wente,2014.0,Pinot Noir,California
2,2,13.8,California Republic,Red,86,15.00,Butter and vanilla notes dominate the jammy fr...,California Republic 2016 Cabernet Sauvignon (C...,California Republic,2016.0,Cabernet Sauvignon,California
3,3,13.8,"Lodi, Central Valley, California, US",Red,86,10.00,An aroma like toasted almonds and wood smoke m...,Collier Creek 2015 Red Wagon Pinot Noir (Lodi),Collier Creek,2015.0,Pinot Noir,California
4,4,13.0,"New Jersey, US",Red,86,27.00,Plum skin and pomegranate lead the nose while ...,DiLuca 2016 Rosso Black Label Red,DiLuca,2016.0,Red Blend,NaN


In [3]:
df.describe(include= 'all')

,Unnamed: 0,Alcohol,Appellation,Category,Points,Price,Review,Wine,Winery,Year,Variety,State
count,4768.000000,4768.000000,4768,4768,4768.000000,4768.000000,4768,4768,4768,4768.000000,4768,3392
unique,NaN,NaN,945,7,NaN,NaN,4678,4676,1520,NaN,117,4
top,NaN,NaN,"Russian River Valley, Sonoma, California, US",Red,NaN,NaN,"Cedar, oak, leather and vanilla dominate the n...",Naggiar 2017 Estate Rosé (Sierra Foothills),Maryhill,NaN,Pinot Noir,California
freq,NaN,NaN,214,3118,NaN,NaN,2,3,38,NaN,1058,2209
mean,2383.500000,13.949778,NaN,NaN,89.340185,42.290271,NaN,NaN,NaN,2015.419044,NaN,NaN
std,1376.547372,0.950407,NaN,NaN,2.784258,32.311230,NaN,NaN,NaN,1.154587,NaN,NaN
min,0.000000,8.000000,NaN,NaN,80.000000,5.000000,NaN,NaN,NaN,2004.000000,NaN,NaN
25%,1191.750000,13.500000,NaN,NaN,87.000000,24.000000,NaN,NaN,NaN,2015.000000,NaN,NaN
50%,2383.500000,14.100000,NaN,NaN,90.000000,35.000000,NaN,NaN,NaN,2015.000000,NaN,NaN
75%,3575.250000,14.500000,NaN,NaN,91.000000,50.000000,NaN,NaN,NaN,2016.000000,NaN,NaN


# Potential target variables to Predict based on data above:
- category: there are 4 unique categories
- state: there are 4 unique states
- variety:117 varieties
- points - below mean or above mean?
- price - below mean or above mean?
- winery - based on the language they used?

In [4]:
len(df)

4768

In [5]:
df.Variety.describe()

count           4768
unique           117
top       Pinot Noir
freq            1058
Name: Variety, dtype: object

In [6]:

# df.isna().sum()

In [7]:
df.columns

Index(['Unnamed: 0', 'Alcohol', 'Appellation', 'Category', 'Points', 'Price',
       'Review', 'Wine', 'Winery', 'Year', 'Variety', 'State'],
      dtype='object')

In [8]:
# region_2 and twitter handle have the most null values so dropping the column
df_cleaned1 = df.drop(['Unnamed: 0', 'Alcohol', 'Appellation', 'Points', 'Price', 'Wine', 'Winery', 'Year', 'State'],axis=1)
df_cleaned1

,Category,Review,Variety
0,Red,"Baked plum, licorice and lavender aromas and f...",Zinfandel
1,Red,Pomegranate and light baking-spice aromas show...,Pinot Noir
2,Red,Butter and vanilla notes dominate the jammy fr...,Cabernet Sauvignon
3,Red,An aroma like toasted almonds and wood smoke m...,Pinot Noir
4,Red,Plum skin and pomegranate lead the nose while ...,Red Blend
5,Red,This is a big ripe red wine moderate in struct...,Pinot Noir
6,Red,This wine is a blend of 48% Cabernet Sauvignon...,Red Blend
7,White,"Honey-dipped pineapple and guava, with a touch...",Sauvignon Blanc
8,White,"Thick and oaky, this estate wine evokes carame...",Chardonnay
9,Red,"With an ashy undercurrent of char and smoke, t...",Sangiovese


In [9]:
list(df_cleaned1.columns.values)

['Category', 'Review', 'Variety']

## Word Vectorization

In [10]:
import pandas as pd
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.manifold import TSNE
from nltk.tokenize import word_tokenize
import nltk
np.random.seed(0)

In [11]:
df_cleaned1.Review

0       Baked plum, licorice and lavender aromas and f...
1       Pomegranate and light baking-spice aromas show...
2       Butter and vanilla notes dominate the jammy fr...
3       An aroma like toasted almonds and wood smoke m...
4       Plum skin and pomegranate lead the nose while ...
5       This is a big ripe red wine moderate in struct...
6       This wine is a blend of 48% Cabernet Sauvignon...
7       Honey-dipped pineapple and guava, with a touch...
8       Thick and oaky, this estate wine evokes carame...
9       With an ashy undercurrent of char and smoke, t...
10      Dusty with highlights of cedar and sage, this ...
11      Yellow apple and sweet oak aromas make for a p...
12      This is a full-bodied, boldly ripe Cabernet Sa...
13      Aromas of ripe blueberry and blackberry mingle...
14      Brawny fruit is overwhelmed by smoke and charr...
15      Pear, butter and vanilla aromas lead to a soft...
16      This is an easygoing, fruity wine that shows s...
17      This h

In [12]:
experiment_line= str(df_cleaned1.Review[2:3].values)
experiment_line

"['Butter and vanilla notes dominate the jammy fruit flavors in this rich and seemingly sweet wine. It is medium bodied, lightly tannic and heavily influenced by oaky aromas and flavors, including baking spices and maple syrup.']"

In [13]:
# experiment_line= df_cleaned4.description[2:3].to_string()

# clean_line = []
# # for word in experiment_line:

pattern = "([a-zA-Z]+(?:'[a-z]+)?)"
line_tokens_raw = nltk.regexp_tokenize(experiment_line, pattern)
print(line_tokens_raw)    
    

['Butter', 'and', 'vanilla', 'notes', 'dominate', 'the', 'jammy', 'fruit', 'flavors', 'in', 'this', 'rich', 'and', 'seemingly', 'sweet', 'wine', 'It', 'is', 'medium', 'bodied', 'lightly', 'tannic', 'and', 'heavily', 'influenced', 'by', 'oaky', 'aromas', 'and', 'flavors', 'including', 'baking', 'spices', 'and', 'maple', 'syrup']


In [14]:
reviews = str(df_cleaned1.Review[0:20].values)

In [15]:
reviews

'[\'Baked plum, licorice and lavender aromas and flavors define this jammy wine, which takes on a sweetness of oak. Thick on the midpalate, it has substantial weight from the tannins and powerfully ripe fruit.\'\n \'Pomegranate and light baking-spice aromas show on the mellow nose of this bottling. The palate is clean with fruit punch and spice-cake flavors wrapped in a fairly mellow texture.\'\n \'Butter and vanilla notes dominate the jammy fruit flavors in this rich and seemingly sweet wine. It is medium bodied, lightly tannic and heavily influenced by oaky aromas and flavors, including baking spices and maple syrup.\'\n "An aroma like toasted almonds and wood smoke meets spiced plum and red-cherry flavors in this medium-bodied wine. It\'s an oaky version that maintains pretty good balance."\n \'Plum skin and pomegranate lead the nose while white pepper and raw oak take on an earthy edge. The soft-bodied palate is awash in silky tannins, with intense flavors of vanilla, oak and green

In [16]:
# categories = ['Red', 'White', 'Orange']

In [17]:
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import *
stemmer = SnowballStemmer("english")

In [45]:
data = df_cleaned1
data

,Category,Review,Variety
0,Red,"Baked plum, licorice and lavender aromas and f...",Zinfandel
1,Red,Pomegranate and light baking-spice aromas show...,Pinot Noir
2,Red,Butter and vanilla notes dominate the jammy fr...,Cabernet Sauvignon
3,Red,An aroma like toasted almonds and wood smoke m...,Pinot Noir
4,Red,Plum skin and pomegranate lead the nose while ...,Red Blend
5,Red,This is a big ripe red wine moderate in struct...,Pinot Noir
6,Red,This wine is a blend of 48% Cabernet Sauvignon...,Red Blend
7,White,"Honey-dipped pineapple and guava, with a touch...",Sauvignon Blanc
8,White,"Thick and oaky, this estate wine evokes carame...",Chardonnay
9,Red,"With an ashy undercurrent of char and smoke, t...",Sangiovese


In [47]:
data.isna().sum()
len(data)

4768

In [48]:
numpy_array = data.as_matrix()
numpy_array

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


array([['Red',
        'Baked plum, licorice and lavender aromas and flavors define this jammy wine, which takes on a sweetness of oak. Thick on the midpalate, it has substantial weight from the tannins and powerfully ripe fruit.',
        'Zinfandel'],
       ['Red',
        'Pomegranate and light baking-spice aromas show on the mellow nose of this bottling. The palate is clean with fruit punch and spice-cake flavors wrapped in a fairly mellow texture.',
        'Pinot Noir'],
       ['Red',
        'Butter and vanilla notes dominate the jammy fruit flavors in this rich and seemingly sweet wine. It is medium bodied, lightly tannic and heavily influenced by oaky aromas and flavors, including baking spices and maple syrup.',
        'Cabernet Sauvignon'],
       ...,
       ['Red',
        'This is a microscopic production, which shows itself to be medium bodied to full, with soft moderate structure. Dried cherry, raspberry and dried citrus round out the approachable, likable fruit-forw

In [49]:
numpy_array.shape()

TypeError: 'tuple' object is not callable

In [51]:
X = numpy_array[:,1:3]
Y = numpy_array[:,0]
X

array([['Baked plum, licorice and lavender aromas and flavors define this jammy wine, which takes on a sweetness of oak. Thick on the midpalate, it has substantial weight from the tannins and powerfully ripe fruit.',
        'Zinfandel'],
       ['Pomegranate and light baking-spice aromas show on the mellow nose of this bottling. The palate is clean with fruit punch and spice-cake flavors wrapped in a fairly mellow texture.',
        'Pinot Noir'],
       ['Butter and vanilla notes dominate the jammy fruit flavors in this rich and seemingly sweet wine. It is medium bodied, lightly tannic and heavily influenced by oaky aromas and flavors, including baking spices and maple syrup.',
        'Cabernet Sauvignon'],
       ...,
       ['This is a microscopic production, which shows itself to be medium bodied to full, with soft moderate structure. Dried cherry, raspberry and dried citrus round out the approachable, likable fruit-forwardness of the layered wine.',
        'Pinot Noir'],
      

In [52]:
X.shape

(4768, 2)

In [53]:
Y.shape

(4768,)

In [54]:
type(Y)

numpy.ndarray

In [58]:
vectorizer = CountVectorizer(
    stop_words='english',
    ngram_range=(2,4768),  #ngram_range=(1, 1) is the default
    dtype='double',
)
data = vectorizer.fit_transform(X.ravel())
data[0]

<1x1397091 sparse matrix of type '<class 'numpy.float64'>'
	with 171 stored elements in Compressed Sparse Row format>

In [61]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline 


text_clf = Pipeline([('vect', CountVectorizer(stop_words='english')),
 ('tfidf', TfidfTransformer()),
 (dtype='double'),
 ('clf', MultinomialNB()),
])

text_clf = text_clf.fit(X_train,Y_train)

predicted = text_clf.predict(X_test)
np.mean(predicted == Y_test)

SyntaxError: invalid syntax (<ipython-input-61-72e8ab3aca1f>, line 12)

In [27]:
#Creates a datafram for word vectors

# pattern = "([a-zA-Z]+(?:'[a-z]+)?)"
# stop_words = set(stopwords.words('english'))

# #figure out how to concatenate eveything back together
# for line in descriptions:
#     article_tokens_raw = nltk.regexp_tokenize(descriptions, pattern)
#     article_tokens = [i.lower() for i in article_tokens_raw]
#     article_tokens_stopped = [w for w in article_tokens if not w in stop_words]
# #     art_stemmed = [stemmer.stem(word) for word in article_tokens_stopped]
#     docs = article_tokens_stopped
#     vec = CountVectorizer()
#     X = vec.fit_transform(docs)
#     df = pd.DataFrame(X.toarray(), columns=vec.get_feature_names())

In [ ]:
import seaborn as sns
fig = plt.figure(figsize = (20,20))
sns.scatterplot(x = X_test, y = Y_test)

In [ ]:
# https://stackoverflow.com/questions/24386489/adding-words-to-scikit-learns-countvectorizers-stop-list